In [100]:
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import plotly.express as px
import shap
import seaborn as sns
from IPython import display
import matplotlib.pyplot as plt
import awswrangler as wr

In [101]:
wr.config.s3_endpoint_url = "http://192.168.1.7:8333"


In [102]:

multilabelDf = wr.s3.read_parquet(
    path=f"s3://multilabel_df/",
    dataset=True,
)

In [103]:
multilabelDf

,vision_bonnet,vision_bumper_front,vision_grille,vision_headlamp_rh,vision_headlamp_lh,vision_door_front_lh,vision_door_front_rh,vision_engine,vision_bumper_rear,vision_misc,...,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear,CaseID,Circumstances_of_Accident
0,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,12657878,Collision- Head to Rear (Insured Hit TP)
1,1,1,0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,12657888,Lost control- Overturned
2,1,1,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,12657890,
3,1,1,1,0,0,0,1,1,0,1,...,1,0,0,0,0,0,0,0,12657900,Collided into animal
4,1,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,12657903,Collision- Head to Rear (Insured Hit TP)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621787,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10147236,
621788,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10936139,Collision- Head to Rear (TP Hit Insured)
621789,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10174259,
621790,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,10174498,


In [104]:
allVisionFeatures = [x for x in multilabelDf.columns if "vision_" in x]
caseFeatures = [
    "Circumstances_of_Accident",
   
]
allInputFeature = caseFeatures + allVisionFeatures
targetCol = [
    x
    for x in multilabelDf.columns
    if x not in allInputFeature and x != "CaseID"
]
len(targetCol)

34

In [105]:
saloonPredDf = pd.read_csv("/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/results/saloon_v1/imgs_pred_output.csv")
hatchBackDf = pd.read_csv("/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/results/hatchback_v1/Hatchback - 5 Dr_imgs_pred_output.csv")
imgPredOutput = pd.concat([saloonPredDf, hatchBackDf])

In [106]:
realTestDataDf = multilabelDf[caseFeatures + targetCol + ["CaseID"]].merge(imgPredOutput, on="CaseID")
realTestDataDf = realTestDataDf.loc[:, ~realTestDataDf.columns.str.contains('^Unnamed')]


In [107]:
realTestDataDf.to_csv("saloon_real_test.csv")

In [108]:
trainDf = multilabelDf[~multilabelDf["CaseID"].isin(realTestDataDf["CaseID"].unique().tolist())]
assert set(realTestDataDf['CaseID'].tolist()).isdisjoint(trainDf['CaseID'].tolist())

In [109]:
X_train = realTestDataDf[allInputFeature]
Y_train = realTestDataDf[targetCol]
X_test = trainDf[allInputFeature]
Y_test = trainDf[targetCol]
train_pool = Pool(X_train, Y_train, cat_features=caseFeatures + allVisionFeatures)
test_pool = Pool(X_test, Y_test, cat_features=caseFeatures + allVisionFeatures)

In [110]:
clf = CatBoostClassifier(
    loss_function="MultiCrossEntropy",
    eval_metric="Accuracy",
    iterations=100,
    # task_type="GPU",
    class_names=targetCol,
)
clf.fit(train_pool, eval_set=test_pool, metric_period=10, plot=True, verbose=5)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.215315
0:	learn: 0.0151429	test: 0.0301043	best: 0.0301043 (0)	total: 470ms	remaining: 46.5s
5:	learn: 0.0213333	test: 0.0204492	best: 0.0301043 (0)	total: 2.47s	remaining: 38.7s
10:	learn: 0.0286334	test: 0.0368984	best: 0.0368984 (10)	total: 4.52s	remaining: 36.5s
15:	learn: 0.0318149	test: 0.0496658	best: 0.0496658 (15)	total: 6.61s	remaining: 34.7s
20:	learn: 0.0337879	test: 0.0542319	best: 0.0542319 (20)	total: 8.68s	remaining: 32.6s
25:	learn: 0.0348731	test: 0.0618517	best: 0.0618517 (25)	total: 10.7s	remaining: 30.4s
30:	learn: 0.0356623	test: 0.0634242	best: 0.0634242 (30)	total: 12.7s	remaining: 28.2s
35:	learn: 0.0369201	test: 0.0636307	best: 0.0636307 (35)	total: 14.7s	remaining: 26.2s
40:	learn: 0.0368461	test: 0.0673176	best: 0.0673176 (40)	total: 16.5s	remaining: 23.8s


In [ ]:
test_predict = clf.predict(X_test)
train_predict = clf.predict(X_train)

In [ ]:
def hamming_score(y_true, y_pred):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list), acc_list

In [ ]:
accuracy = eval_metric(Y_test.values, test_predict, "Accuracy")[0]
print(f"Accuracy: {accuracy}")

Accuracy: 0.08943044671352011


In [ ]:
subsetAcc2, acc_list = hamming_score(Y_test.values, test_predict)
subsetAcc2

0.4673418589373764

In [ ]:
avgPrecision = 0
avgRecall = 0
avgF1 = 0
allRecall = []
allPrecision = []
clsOrder = clf.classes_
allF1 = []
allSupport = []
allAcc = []
allBalanceAcc = []
highRecallCount = 0
lowRecallCount = 0
recallLowThreshold = 0.2
highRecallThreshold = 0.6
for metric in ("Precision", "Recall", "F1", "Accuracy"):
    # print(metric)
    values = eval_metric(Y_test.values, test_predict, metric)
    if metric == "Precision":
        avgPrecision = np.mean(values)
    elif metric == "Recall":
        avgRecall = np.mean(values)

    elif metric == "F1":
        avgF1 = np.mean(values)
    for cls, value in zip(clf.classes_, values):
        
        # print(f"class={cls}: {value:.4f}")
        if metric == "Recall":
            allRecall.append(value)
            if value <= recallLowThreshold:
                lowRecallCount += 1
            elif value >= highRecallThreshold:
                highRecallCount += 1
        elif(metric == "Precision"):
            allPrecision.append(value)
        elif(metric == "F1"):
            allF1.append(value)
        elif metric == "Accuracy":
            allAcc.append(value)
    print()
print(f"Avg Precision : {avgPrecision}")
print(f"Avg Recall : {avgRecall}")
print(f"Avg F1 : {avgF1}")
print(f"Low Recall Count : {lowRecallCount}")
print(f"High Recall Count : {highRecallCount}")
print(f"all acc : {allAcc}")

summaryPerfDf = pd.DataFrame({
    "recall" : allRecall,
    "precision" : allPrecision,
    "f1" : allF1,
    "part" :  clsOrder,
    "avgRecall" : avgRecall,
    "avgPrecision" : avgPrecision,
    "avgF1" : avgF1,


})





Avg Precision : 0.858405064624816
Avg Recall : 0.4555272874779443
Avg F1 : 0.5693233638331069
Low Recall Count : 6
High Recall Count : 12
all acc : [0.08943044671352011]


In [ ]:
avgDf = pd.DataFrame(
    {
        "AvgRecall" : [avgRecall],
        "AvgPrecision" : [avgPrecision],
        "AvgF1" : [avgF1],
        "ExactMatch" : [accuracy],
        "SubsetAccuracy" : [subsetAcc2],


    }
).transpose().reset_index()
# fig = px.bar(avgDf, x = 'part', y = 'recall')
fig = px.bar(avgDf, x = 'index', y = 0, text_auto=True)

fig

In [ ]:
fig = px.bar(summaryPerfDf, x = 'part', y = 'recall')

fig.add_hline(y=avgRecall, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgRecall, 1), 0.2, 0.5, 0.7],
    )
)
fig

In [ ]:
fig = px.bar(summaryPerfDf, x = 'part', y = 'precision')

fig.add_hline(y=avgRecall, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgRecall, 1), 0.2, 0.5, 0.7],
    )
)
fig

In [ ]:
fig = px.bar(summaryPerfDf, x = 'part', y = 'f1')

fig.add_hline(y=avgRecall, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgRecall, 1), 0.2, 0.5, 0.7],
    )
)
fig